In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np


In [3]:
ano = "2022"

In [67]:
from data_master_eng_ml.utils.twitch_api import build_query, fetch_data_with_pagination
from data_master_eng_ml.utils.api_transformations import (
    fetch_game_release_dates,
    fetch_involved_companies,
    fetch_companies_info,
    fetch_multiplayer_modes,
    fetch_game_info,
    batch_fetch_age_classifications
)
from data_master_eng_ml.utils.helpers import map_age_classifications,generate_dummies
from data_master_eng_ml.config import URL_TWITCH_BASE

In [5]:
# Fighting - 4
# Racing - 10
# 6	Full Release

In [7]:
single = False
year = 2022

In [8]:

data_frame_games_find = fetch_game_release_dates(single=single,year=year)

In [9]:
games_id = list(set(data_frame_games_find["game"].to_list()))

In [10]:
games_id_list = ",".join(map(str, games_id))

In [11]:
len(games_id_list)

11470

In [17]:
# Chamada da função com paginação
data_frame_companies_find = fetch_involved_companies(games_id_list)
data_frame_companies_find
company_id_list = list(set(data_frame_companies_find["company"].to_list()))


In [13]:
data_frame_companies_find

,id,company,created_at,developer,game,porting,publisher,supporting,updated_at,checksum
0,257101,22440,1709880472,True,227913,False,True,False,1710125858,c611d8b7-c58c-114f-d621-30f32ff9ae85
1,185685,12177,1662098659,False,27366,False,True,False,1662101178,c558368e-2c76-3a19-340d-6fc44364338d
2,185686,5781,1662102285,True,215093,False,False,False,1662102775,2ab2cfed-38df-3232-4cc2-564796ff4ec0
3,185687,43005,1662102285,False,215093,False,True,False,1662102775,43d196a9-2b8d-b982-ee85-6219f9d3237a
4,258679,53624,1710720793,True,219363,False,True,False,1710722577,aec26cb7-5a2c-a090-d8c9-f4e25f3714ae
...,...,...,...,...,...,...,...,...,...,...
1608,285677,24004,1725563229,True,207093,False,False,False,1725563569,6dc8726d-2389-3ef7-9171-17d6547add39
1609,285699,33166,1725569281,False,173902,False,True,False,1725570717,b6f132b1-c755-d02c-1543-42736e03884c
1610,285697,41282,1725569280,True,173902,False,False,False,1725570717,552b192b-b770-01d7-70aa-204dae6150a9
1611,285698,8891,1725569281,False,173902,False,True,False,1725570717,e27027f4-c7e7-6338-fdf4-4f1d35c8dc24


In [20]:
data_frame_companies = fetch_companies_info(company_id_list)

In [21]:
data_frame_companies

,id,slug,start_date,games_developed,has_parents,games_published,continent_name
0,53338,soner-cakir,NaN,4,0,4,Unknown
1,56535,fe5-ai,NaN,1,0,1,Unknown
2,23139,nutaku-publishing,1.450310e+09,1,0,13,North America
3,54980,goat,NaN,-1,0,1,Unknown
4,52715,ben-big-game-studio,NaN,1,0,1,Unknown
...,...,...,...,...,...,...,...
1209,26,square-enix,1.049155e+09,187,0,962,Asia
1210,23125,playism,1.304208e+09,-1,1,139,Asia
1211,16437,team17,6.626016e+08,78,0,170,Europe
1212,82,atari,7.084800e+07,244,1,337,North America


In [16]:
# Inner Join (default)
companies_join = pd.merge(
    data_frame_companies_find,
    data_frame_companies,
    left_on="company",
    right_on="id",
    how="inner",
)

In [17]:
df_companies_final = companies_join[
    ["game", "games_developed", "has_parents", "games_published", "continent_name"]
]

In [24]:
data_frame_multiplayer_modes = fetch_multiplayer_modes(games_id_list)
data_frame_multiplayer_modes

,id,campaigncoop,game,lancoop,offlinecoop,offlinecoopmax,offlinemax,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,5456,False,13205,False,False,0.0,0.0,True,4.0,0.0,False
1,9819,False,116599,False,False,NaN,NaN,True,2.0,2.0,False
2,10346,False,131957,False,False,NaN,NaN,False,NaN,NaN,False
3,10862,False,105594,False,False,NaN,NaN,True,NaN,NaN,False
4,11594,False,72,True,True,2.0,0.0,True,2.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...
135,29591,False,159116,False,False,NaN,NaN,True,NaN,22.0,False
136,28660,False,231049,False,True,4.0,4.0,False,NaN,NaN,True
137,28661,False,231049,False,True,4.0,4.0,False,NaN,NaN,True
138,28847,False,307264,False,False,NaN,2.0,False,NaN,2.0,False


In [57]:
data_frame_games = fetch_game_info(games_id_list)

In [58]:
data_frame_games

,id,age_ratings,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target
0,71,"[984, 4395, 47678, 47679, 47680, 47681, 47682]",Portal,[first-person],"[mobile, pc, modern_console]",shooter,[single-player],False,1
1,72,"[11721, 32022, 47684, 47685, 47687, 125644, 14...",Portal 2,[first-person],"[pc, modern_console]",shooter,"[multiplayer, co-operative, single-player, spl...",False,1
2,4250,"[65229, 142924, 179089]",Kingdom: The Far Reaches,[third-person],"[less_common_portable_console, unknown_platfor...",adventure,[single-player],False,0
3,6614,"[8996, 32717, 99650]",Zero Escape: Virtue's Last Reward,"[text, first-person]","[portable_console, pc, modern_console]",visual-novel,[single-player],False,1
4,7046,"[79834, 91077, 97910]",Factorio,[bird-view-slash-isometric],"[pc, modern_console]",strategy,"[multiplayer, co-operative, single-player]",False,1
...,...,...,...,...,...,...,...,...,...
1643,61080,"[45079, 59140, 97948, 119399, 171971, 171972, ...",Ninja Village,[bird-view-slash-isometric],"[mobile, pc, modern_console]",strategy,[single-player],False,0
1644,132058,"[82252, 102582, 111285, 114570]",Rogue Legacy 2,[side-view],"[pc, modern_console]",role-playing-rpg,[single-player],False,1
1645,143612,"[56633, 90852, 118403, 180772, 180773, 180774,...",Neon White,"[text, first-person]","[pc, modern_console]",shooter,[single-player],False,1
1646,135929,"[78029, 78068, 130323, 144094, 144095, 144096]",Apico,[bird-view-slash-isometric],"[pc, modern_console]",adventure,"[multiplayer, co-operative, single-player]",False,0


In [59]:
# Faz o batch fetch das classificações etárias
unique_age_ratings = data_frame_games['age_ratings'].explode().dropna().unique().tolist()

In [60]:
age_ratings_df = batch_fetch_age_classifications(unique_age_ratings)

In [62]:
# Mapeia as classificações etárias para o DataFrame de jogos
data_frame_games = map_age_classifications(data_frame_games, age_ratings_df)

In [64]:
companies_join = pd.merge(data_frame_companies_find, data_frame_companies, left_on="company", right_on="id", how="inner")

df_companies_final = companies_join[["game", "games_developed", "has_parents", "games_published", "continent_name"]]

In [65]:
# Merge com dados das empresas e modos multiplayer
df_companies_games = pd.merge(df_companies_final, data_frame_games_find[["game", "region_name"]].drop_duplicates(subset="game"), on="game", how="inner")
df_companies_games_multiplayer = pd.merge(df_companies_games.drop_duplicates(subset="game"), data_frame_multiplayer_modes[["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]], on="game", how="left")

# Combina tudo em um único DataFrame final
df_join = pd.merge(data_frame_games, df_companies_games_multiplayer, left_on="id", right_on="game", how="left").drop(columns=["game"]).drop_duplicates(subset="id")


In [66]:
df_join

,id,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,region_name,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,71,Portal,[first-person],"[mobile, pc, modern_console]",shooter,[single-player],False,1,15+,125.0,0.0,1353.0,North America,worldwide,NaN,NaN,NaN,NaN
1,72,Portal 2,[first-person],"[pc, modern_console]",shooter,"[multiplayer, co-operative, single-player, spl...",False,1,12+,86.0,0.0,95.0,North America,worldwide,True,2.0,0.0,True
6,4250,Kingdom: The Far Reaches,[third-person],"[less_common_portable_console, unknown_platfor...",adventure,[single-player],False,0,12+,4.0,0.0,39.0,Unknown,worldwide,NaN,NaN,NaN,NaN
7,6614,Zero Escape: Virtue's Last Reward,"[text, first-person]","[portable_console, pc, modern_console]",visual-novel,[single-player],False,1,18+,1.0,0.0,132.0,Europe,worldwide,NaN,NaN,NaN,NaN
8,7046,Factorio,[bird-view-slash-isometric],"[pc, modern_console]",strategy,"[multiplayer, co-operative, single-player]",False,1,12+,2.0,0.0,2.0,Europe,worldwide,True,65535.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,296814,Genesis II,[bird-view-slash-isometric],"[portable_console, unknown_platforms_name]",shooter,[single-player],False,0,No Rating,6.0,0.0,-1.0,Europe,worldwide,NaN,NaN,NaN,NaN
1677,230728,Four Corners for Playdate,[unknown_player_perspectives],[unknown_platforms_name],arcade,[unknown_game_mode],False,0,No Rating,9.0,0.0,9.0,Unknown,worldwide,NaN,NaN,NaN,NaN
1678,294670,Ad Astra,"[third-person, first-person]",[pc],adventure,"[multiplayer, co-operative, single-player]",False,0,No Rating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,229164,Cubic Light,[unknown_player_perspectives],[modern_console],indie,[single-player],False,0,3+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Gera variáveis dummy
df_join = generate_dummies(df_join, ["platforms_name", "game_modes_name", "player_perspective_name"])

# Adiciona flag para lançamento global
df_join["has_global_launch"] = np.where(df_join["region_name"] == "worldwide", 1, 0)
df_join = df_join.drop(columns=["region_name"])

In [69]:
df_join

,id,name,genres_first,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,...,unknown_game_mode,auditory,bird_view_slash_isometric,first_person,side_view,text,third_person,unknown_player_perspectives,virtual_reality,has_global_launch
0,71,Portal,shooter,False,1,15+,125.0,0.0,1353.0,North America,...,0,0,0,1,0,0,0,0,0,1
1,72,Portal 2,shooter,False,1,12+,86.0,0.0,95.0,North America,...,0,0,0,1,0,0,0,0,0,1
2,4250,Kingdom: The Far Reaches,adventure,False,0,12+,4.0,0.0,39.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
3,6614,Zero Escape: Virtue's Last Reward,visual-novel,False,1,18+,1.0,0.0,132.0,Europe,...,0,0,0,1,0,1,0,0,0,1
4,7046,Factorio,strategy,False,1,12+,2.0,0.0,2.0,Europe,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,296814,Genesis II,shooter,False,0,No Rating,6.0,0.0,-1.0,Europe,...,0,0,1,0,0,0,0,0,0,1
1268,230728,Four Corners for Playdate,arcade,False,0,No Rating,9.0,0.0,9.0,Unknown,...,1,0,0,0,0,0,0,1,0,1
1269,294670,Ad Astra,adventure,False,0,No Rating,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,1,0,0,0
1270,229164,Cubic Light,indie,False,0,3+,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0


In [40]:
df_join.to_csv(f'twitch_api_data_{ano}.csv',header=True,index=False)